In [18]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [19]:
#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_mean(var))#最小值
        tf.summary.histogram('histogram',var)#直方图

with tf.name_scope('input'):
    x=tf.placeholder(tf.float32,shape=[None,784],name='x_input')
    y=tf.placeholder(tf.float32,shape=[None,10],name='y_input')

def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)#正态分布
    return tf.Variable(initial)
    
def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv(x,W):

#x:[batch,in_height,in_width,in_channels]
#w:[height,width,in_channesl,out_channels]
#strides:strides[0]=strides[3]=1,strides[1]=x步长，strides[2]=y步长
#paddings:SAME,VALID

    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#shape[batch_size,in_height,in_width,in_channels]
x_image=tf.reshape(x,shape=[-1,28,28,1])

with tf.name_scope('conv'):
    with tf.name_scope('conv1'):
        conv1=conv(x_image,weight_variable([5,5,1,32]))#5*5的采样窗口，32个卷积核从一个平面提取特征
        bias1=bias_variable([32])#每个卷积核一个偏值
        relu1=tf.nn.relu(conv1+bias1)
        pool1=max_pool_2x2(relu1)
        
    with tf.name_scope('conv2'):
        conv2=conv(pool1,weight_variable([5,5,32,64]))
        bias2=bias_variable([64])
        relu2=tf.nn.relu(conv2+bias2)
        pool2=max_pool_2x2(relu2)
        
    with tf.name_scope('full_connect'):
        with tf.name_scope('full_connect1'):
            pool2_flat=tf.reshape(pool2,[-1,7*7*64])
            W_fc1=weight_variable([7*7*64,1024])
            b_fc1=bias_variable([1024])
            fc1=tf.nn.relu(tf.matmul(pool2_flat,W_fc1)+b_fc1)
            
        with tf.name_scope('dropout'):
            keep_prob=tf.placeholder(tf.float32)
            fc1_dropout=tf.nn.dropout(fc1,keep_prob)
        
        with tf.name_scope('full_connect2'):
            W_fc2=weight_variable([1024,10])
            b_fc2=bias_variable([10])
        with tf.name_scope('prediction'):
            prediction=tf.nn.softmax(tf.matmul(fc1_dropout,W_fc2)+b_fc2,name='prediction')
            
with tf.name_scope('loss'):
    cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss', cross_entropy)
with tf.name_scope('train_step'):
    train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
with tf.name_scope('correct_prediction'):
    correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
with tf.name_scope('accuracy'):
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('accuracy',accuracy)
                                 
merge=tf.summary.merge_all()

In [20]:
mnist=input_data.read_data_sets('./MNIST',one_hot=True)
batch_size=100
n_batch=mnist.train.num_examples//batch_size
init=tf.global_variables_initializer()

Extracting ./MNIST\train-images-idx3-ubyte.gz
Extracting ./MNIST\train-labels-idx1-ubyte.gz
Extracting ./MNIST\t10k-images-idx3-ubyte.gz
Extracting ./MNIST\t10k-labels-idx1-ubyte.gz


In [21]:
with tf.Session()as sess:
    sess.run(init)
    writer=tf.summary.FileWriter('./logs',sess.graph)
    for epoch in range(50):
        for _ in range(n_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            summary,_=sess.run([merge,train_step],feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})

        writer.add_summary(summary,epoch)
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print('Iter'+str(epoch)+',accuracy'+str(acc))

KeyboardInterrupt: 